In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE101710"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE101710"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE101710.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE101710.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE101710.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analysis of Influenza vaccine response in Young and Old - Year 5"
!Series_summary	"We profiled gene expression from a stratified cohort of subjects to define influenza vaccine response in Young and Old"
!Series_overall_design	"Differential gene expression by human PBMCs from Healthy Adults receiving Influenza Vaccination (Fluvirin, Novartis). Healthy adults (older >65, younger 21-30 years) were recruited at seasonal Influenza Vaccination clinics organized by Yale University Health Services during October to December of 2014 – 2015 seasons. With informed consent, healthy individuals were recruited as per a protocol approved by Human Investigations Committee of the Yale University School of Medicine. Each subject was evaluated by a screening questionnaire determining self-reported demographic information, height, weight, medications and comorbid conditions. Participants with acute illness two weeks prior to vaccination were excluded 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
from typing import Optional, Any, Dict, Callable

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from Illumina HT12-V4.0 BeadChip
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For height (our trait): The background info mentions height was collected in screening questionnaire
# But looking at the sample characteristics, there's no direct height data
trait_row = None  # Height data is not available in the sample characteristics

# For age: Age group is available in row 1 
age_row = 1  # Contains "age group: Older", "age group: Frail", "age group: Young"

# For gender: No gender information in the sample characteristics
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion

# Define conversion functions for each variable
def convert_trait(value: str) -> Optional[float]:
    """Convert height data to float (continuous). Not used in this dataset."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_age(value: str) -> Optional[int]:
    """Convert age group to binary (0 for Young, 1 for Older/Frail)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'young' in value:
        return 0
    elif 'older' in value or 'frail' in value:
        return 1
    else:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male). Not used in this dataset."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# is_trait_available is False since trait_row is None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None (no height data available)
# If we had trait data, we would execute:
# if trait_row is not None:
#     # Assuming clinical_data is loaded from a previous step
#     clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
#     selected_clinical_df = geo_select_clinical_features(
#         clinical_df=clinical_data,
#         trait=trait,
#         trait_row=trait_row,
#         convert_trait=convert_trait,
#         age_row=age_row,
#         convert_age=convert_age if age_row is not None else None,
#         gender_row=gender_row,
#         convert_gender=convert_gender if gender_row is not None else None
#     )
#     
#     # Preview the dataframe
#     preview = preview_df(selected_clinical_df)
#     print(preview)
#     
#     # Save the clinical data
#     os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
#     selected_clinical_df.to_csv(out_clinical_data_file, index=False)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 46892 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# These are Illumina microarray probe identifiers (ILMN_*), not human gene symbols.
# They need to be mapped to official gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE101710
Line 6: !Series_title = Gene expression analysis of Influenza vaccine response in Young and Old - Year 5
Line 7: !Series_geo_accession = GSE101710
Line 8: !Series_status = Public on May 26 2019
Line 9: !Series_submission_date = Jul 20 2017
Line 10: !Series_last_update_date = Jul 25 2021
Line 11: !Series_pubmed_id = 30239628
Line 12: !Series_pubmed_id = 32060136
Line 13: !Series_summary = We profiled gene expression from a stratified cohort of subjects to define influenza vaccine response in Young and Old
Line 14: !Series_overall_design = Differential gene expression by human PBMCs from Healthy Adults receiving Influenza Vaccination (Fluvirin, Novartis). Healthy adults (older >65, younger 21-30 yea


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the correct columns for mapping
# From the gene annotation preview, we can see:
# - 'ID' contains the Illumina probe identifiers (ILMN_*)
# - 'Symbol' contains gene symbols

# 2. Create gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Created gene mapping with {len(gene_mapping)} rows")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
# This handles the many-to-many relationship between probes and genes
try:
    print("\nApplying gene mapping to convert probe-level data to gene-level data...")
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    
    print(f"Successfully converted to gene-level data with {len(gene_data)} genes")
    print("First 10 gene symbols:")
    print(gene_data.index[:10])
    
    # Check if the dataset has non-empty gene data
    if gene_data.empty:
        print("Warning: No genes were successfully mapped")
        is_gene_available = False
    else:
        is_gene_available = True
        
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping: {e}")
    is_gene_available = False


Created gene mapping with 44837 rows
Gene mapping preview:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

Applying gene mapping to convert probe-level data to gene-level data...
Successfully converted to gene-level data with 21344 genes
First 10 gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Height/gene_data/GSE101710.csv


### Step 7: Data Normalization and Linking

In [8]:
import numpy as np
import os
import gzip

# 1. Extract gene expression data using the alternative approach that worked in Step 3
with gzip.open(matrix_file, 'rt') as file:
    # Find the start of the data
    for line in file:
        if '!series_matrix_table_begin' in line:
            break
            
    # Read the headers and data
    gene_data = pd.read_csv(file, sep='\t', index_col=0)

# Check if we have gene data before proceeding
if gene_data.empty:
    print("No gene expression data found in the matrix file.")
    is_gene_available = False
else:
    is_gene_available = True
    print(f"Gene data shape before normalization: {gene_data.shape}")

    # Normalize gene symbols using the NCBI Gene database information
    try:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
        
        # Save the normalized gene data to the output file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error normalizing gene data: {e}")
        is_gene_available = False
        normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
if is_gene_available:
    sample_ids = gene_data.columns
    minimal_clinical_df = pd.DataFrame(index=sample_ids)
    minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

    # If we have age and gender data from Step 2, add those columns
    if age_row is not None:
        minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

    if gender_row is not None:
        minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

    minimal_clinical_df.index.name = 'Sample'

    # Save this minimal clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    minimal_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Create a linked dataset 
    if is_gene_available and normalized_gene_data is not None:
        linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
        linked_data.index.name = 'Sample'
        print(f"Linked data shape: {linked_data.shape}")
    else:
        linked_data = minimal_clinical_df
        print("No gene data to link with clinical data.")
else:
    # Create a minimal dataframe with just the trait for the validation step
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No gene data available, creating minimal dataframe for validation.")

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")

Gene data shape before normalization: (46893, 79)
Gene data shape after normalization: (0, 79)
Normalized gene data saved to ../../output/preprocess/Height/gene_data/GSE101710.csv
Clinical data saved to ../../output/preprocess/Height/clinical_data/GSE101710.csv
Linked data shape: (79, 2)
Abnormality detected in the cohort: GSE101710. Preprocessing failed.
Dataset usability: False
Dataset does not contain Height data and cannot be used for association studies.
